In [ ]:
"""
Author: Liam Bogucki
Email: lboguck@uwo.ca
First Written: Tuesday, July 23, 2024
Last Modified: Thursday, September 4, 2025
Program Purpose: To create the map of what Ahlstrohm's biomes look like with the new MODIS and climate data. Also,
to calculate the IAV contributions of the regions and plot them.
"""

In [ ]:
#Importing appropriate libraries
import numpy as np
import matplotlib.pyplot as plt
import glob
import netCDF4 as nc
import statistics as stats
from collections import OrderedDict

# Changing over the font
plt.rcParams['font.family'] = 'Times New Roman'

In [ ]:
#This block verifies what the MODIS data looks like
modis_array = np.load("MODIS_Map.npy")

plt.subplots(figsize=(20,10))
plt.imshow(modis_array)
plt.colorbar()

In [ ]:
# This block handles the calculation of the IAV contriution of the various biomes under consideration

# Loading area file for adjustments
area_file = np.loadtxt('halfdeg_grid_area.dat')

#Tiling to match the 52 years of flux data
combined_maps_52 = np.tile(modis_array, (52,1,1))

# Ordered dictionaries to hold the IAV values
all_tropical_IAV = OrderedDict() # Tropical IAV
all_extra_tropical_IAV = OrderedDict() # Extra-Tropical IAV
all_dryland_IAV = OrderedDict() # Dryland IAV
all_arctic_shrub_IAV = OrderedDict() # Arctic-shrub IAV
all_grass_crops_IAV = OrderedDict() # Grass + Crops IAV
all_sparsely_IAV = OrderedDict() # Sparsely vegetated IAV

#Using glob to load the files based on the starting pattern and wildcard
pattern = "fco2*"
files = glob.glob(pattern)
name_remove = ["fco2_", "_Dec2022-ext3_1970-2021_yearlymean_XYT.nc"]

for file in files:
    # Getting a name for the files
    file_name = file
    for remove in name_remove:
        file_name = file_name.replace(remove, "")
    file_name = file_name.replace("-", "_").lower()

    # Loading the file and grabbing all of the terrestrial fluxes.
    dataset = nc.Dataset(file, 'r')
    terrestrial_flux = dataset.variables['Terrestrial_flux'][:].astype(np.float64) * 8760  # Convert units to KgC/m2/year from KgC/m2/hr
    terrestrial_flux *= area_file  # Apply the grid cell area adjustment so that fluxes are based on the grid cell size. 10^7 to 10^9
    terrestrial_flux *= 1e-12 #This is the correct value.
    terrestrial_flux = np.flip(terrestrial_flux, axis=1) # Flipping the terrestrial flux data so that it is correctly oriented.
    dataset.close()

    # Removing the improperly high values from the Classic dataset
    if file_name == "classic_s3":
        threshold = 1.86e+34 # Very high value close to fill value but not quite there.
        terrestrial_flux[terrestrial_flux > threshold] = 0 # Replacing with zero.

    # Removing the water values from the yibs that are not blocked out
    if file_name == "yibs_s3":
        data_modis = np.load("Modis_0.5Deg_Type3_Mode.npy")
        data_modis_3d= data_modis[np.newaxis, :, :] # Making the data 3d
        data_modis_52= np.repeat(data_modis_3d, 52, axis=0)
        mask = data_modis_52 != 0
        terrestrial_flux = np.ma.masked_array(terrestrial_flux, mask=~mask)

    # Find the tropical flux each year.
    terrestrial_flux_tropical = np.ma.array(terrestrial_flux, mask = (combined_maps_52 != 1)) # ==1 for the tropical region
    yearly_fluxes_tropical = terrestrial_flux_tropical.sum(axis=(1,2)) # Yearly flux from the tropical regions
    yearly_fluxes_tropical_list = yearly_fluxes_tropical.tolist()
    # Calculate yearly tropical flux anomaly.
    tropical_average_flux = sum(yearly_fluxes_tropical_list) / len(yearly_fluxes_tropical_list) # Sum annual fluxes / # years
    yearly_tropical_flux_anomaly = [] 
    for yearly_flux in yearly_fluxes_tropical_list:
        anomaly_to_add = yearly_flux - tropical_average_flux # Yearly mean - mean of yearly global fluxes... How we find anomaly
        yearly_tropical_flux_anomaly.append(anomaly_to_add)

    # Testing the tropical flux anomaly values.
    print(f"{file_name} has the following yearly tropical flux anomalies: {yearly_tropical_flux_anomaly} \n")

    # Find the extra_tropical flux each year.
    terrestrial_flux_extra_tropical = np.ma.array(terrestrial_flux, mask = (combined_maps_52 != 2)) # ==2 for the extra_tropical region
    yearly_fluxes_extra_tropical = terrestrial_flux_extra_tropical.sum(axis=(1,2)) # Yearly flux from the extra_tropical regions
    yearly_fluxes_extra_tropical_list = yearly_fluxes_extra_tropical.tolist()
    # Calculate yearly extra_tropical flux anomaly.
    extra_tropical_average_flux = sum(yearly_fluxes_extra_tropical_list) / len(yearly_fluxes_extra_tropical_list) # Sum annual fluxes / # years
    yearly_extra_tropical_flux_anomaly = [] 
    for yearly_flux in yearly_fluxes_extra_tropical_list:
        anomaly_to_add = yearly_flux - extra_tropical_average_flux # Yearly mean - mean of yearly global fluxes... How we find anomaly
        yearly_extra_tropical_flux_anomaly.append(anomaly_to_add)

    # Find the dryland flux each year.
    terrestrial_flux_dryland = np.ma.array(terrestrial_flux, mask = (combined_maps_52 != 3)) # ==3 for the dryland region
    yearly_fluxes_dryland = terrestrial_flux_dryland.sum(axis=(1,2)) # Yearly flux from the dryland regions
    yearly_fluxes_dryland_list = yearly_fluxes_dryland.tolist()
    # Calculate yearly dryland flux anomaly.
    dryland_average_flux = sum(yearly_fluxes_dryland_list) / len(yearly_fluxes_dryland_list) # Sum annual fluxes / # years
    yearly_dryland_flux_anomaly = [] 
    for yearly_flux in yearly_fluxes_dryland_list:
        anomaly_to_add = yearly_flux - dryland_average_flux # Yearly mean - mean of yearly global fluxes... How we find anomaly
        yearly_dryland_flux_anomaly.append(anomaly_to_add)

    # Find the arctic_shrub flux each year.
    terrestrial_flux_arctic_shrub = np.ma.array(terrestrial_flux, mask = (combined_maps_52 != 4)) # ==4 for the arctic_shrub region
    yearly_fluxes_arctic_shrub = terrestrial_flux_arctic_shrub.sum(axis=(1,2)) # Yearly flux from the arctic_shrub regions
    yearly_fluxes_arctic_shrub_list = yearly_fluxes_arctic_shrub.tolist()
    # Calculate yearly arctic_shrub flux anomaly.
    arctic_shrub_average_flux = sum(yearly_fluxes_arctic_shrub_list) / len(yearly_fluxes_arctic_shrub_list) # Sum annual fluxes / # years
    yearly_arctic_shrub_flux_anomaly = [] 
    for yearly_flux in yearly_fluxes_arctic_shrub_list:
        anomaly_to_add = yearly_flux - arctic_shrub_average_flux # Yearly mean - mean of yearly global fluxes... How we find anomaly
        yearly_arctic_shrub_flux_anomaly.append(anomaly_to_add)

    # Find the grass_crops flux each year.
    terrestrial_flux_grass_crops = np.ma.array(terrestrial_flux, mask = (combined_maps_52 != 5)) # ==5 for the grass_crops region
    yearly_fluxes_grass_crops = terrestrial_flux_grass_crops.sum(axis=(1,2)) # Yearly flux from the grass_crops regions
    yearly_fluxes_grass_crops_list = yearly_fluxes_grass_crops.tolist()
    # Calculate yearly grass_crops flux anomaly.
    grass_crops_average_flux = sum(yearly_fluxes_grass_crops_list) / len(yearly_fluxes_grass_crops_list) # Sum annual fluxes / # years
    yearly_grass_crops_flux_anomaly = [] 
    for yearly_flux in yearly_fluxes_grass_crops_list:
        anomaly_to_add = yearly_flux - grass_crops_average_flux # Yearly mean - mean of yearly global fluxes... How we find anomaly
        yearly_grass_crops_flux_anomaly.append(anomaly_to_add)

    # Find the sparsely flux each year.
    terrestrial_flux_sparsely = np.ma.array(terrestrial_flux, mask = (combined_maps_52 != 6)) # ==6 for the sparsely region
    yearly_fluxes_sparsely = terrestrial_flux_sparsely.sum(axis=(1,2)) # Yearly flux from the sparsely regions
    yearly_fluxes_sparsely_list = yearly_fluxes_sparsely.tolist()
    # Calculate yearly sparsely flux anomaly.
    sparsely_average_flux = sum(yearly_fluxes_sparsely_list) / len(yearly_fluxes_sparsely_list) # Sum annual fluxes / # years
    yearly_sparsely_flux_anomaly = [] 
    for yearly_flux in yearly_fluxes_sparsely_list:
        anomaly_to_add = yearly_flux - sparsely_average_flux # Yearly mean - mean of yearly global fluxes... How we find anomaly
        yearly_sparsely_flux_anomaly.append(anomaly_to_add)


    # Calculating the yearly_global_flux_anomaly now that we have the flux anomaly of the 6 regions
    yearly_global_flux_anomaly = list()
    
    for i in range(52):
        yearly_global_flux_anomaly.append(yearly_tropical_flux_anomaly[i] + yearly_extra_tropical_flux_anomaly[i] + yearly_dryland_flux_anomaly[i] + yearly_arctic_shrub_flux_anomaly[i] + yearly_grass_crops_flux_anomaly[i] + yearly_sparsely_flux_anomaly[i])

    # Calculating the tropical IAV values.
    top_sum_tropical = 0
    for i in range(52): # Summing for the 52 years.
        top_sum_tropical += (yearly_tropical_flux_anomaly[i] * abs(yearly_global_flux_anomaly[i])) / yearly_global_flux_anomaly[i]
    bottom_sum_tropical = 0
    for i in range(52): # Summing for the 52 years.
        bottom_sum_tropical += abs(yearly_global_flux_anomaly[i])
    # Adding the tropical IAV for that model to the master dictionary.
    all_tropical_IAV[file_name] = top_sum_tropical / bottom_sum_tropical
    
    # Calculating the extra_tropical IAV values.
    top_sum_extra_tropical = 0
    for i in range(52): # Summing for the 52 years.
        top_sum_extra_tropical += (yearly_extra_tropical_flux_anomaly[i] * abs(yearly_global_flux_anomaly[i])) / yearly_global_flux_anomaly[i]
    bottom_sum_extra_tropical = 0
    for i in range(52): # Summing for the 52 years.
        bottom_sum_extra_tropical += abs(yearly_global_flux_anomaly[i])
    # Adding the extra_tropical IAV for that model to the master dictionary.
    all_extra_tropical_IAV[file_name] = top_sum_extra_tropical / bottom_sum_extra_tropical

    # Calculating the dryland IAV values.
    top_sum_dryland = 0
    for i in range(52): # Summing for the 52 years.
        top_sum_dryland += (yearly_dryland_flux_anomaly[i] * abs(yearly_global_flux_anomaly[i])) / yearly_global_flux_anomaly[i]
    bottom_sum_dryland = 0
    for i in range(52): # Summing for the 52 years.
        bottom_sum_dryland += abs(yearly_global_flux_anomaly[i])
    # Adding the dryland IAV for that model to the master dictionary.
    all_dryland_IAV[file_name] = top_sum_dryland / bottom_sum_dryland

    # Calculating the arctic_shrub IAV values.
    top_sum_arctic_shrub = 0
    for i in range(52): # Summing for the 52 years.
        top_sum_arctic_shrub += (yearly_arctic_shrub_flux_anomaly[i] * abs(yearly_global_flux_anomaly[i])) / yearly_global_flux_anomaly[i]
    bottom_sum_arctic_shrub = 0
    for i in range(52): # Summing for the 52 years.
        bottom_sum_arctic_shrub += abs(yearly_global_flux_anomaly[i])
    # Adding the arctic_shrub IAV for that model to the master dictionary.
    all_arctic_shrub_IAV[file_name] = top_sum_arctic_shrub / bottom_sum_arctic_shrub

    # Calculating the grass_crops IAV values.
    top_sum_grass_crops = 0
    for i in range(52): # Summing for the 52 years.
        top_sum_grass_crops += (yearly_grass_crops_flux_anomaly[i] * abs(yearly_global_flux_anomaly[i])) / yearly_global_flux_anomaly[i]
    bottom_sum_grass_crops = 0
    for i in range(52): # Summing for the 52 years.
        bottom_sum_grass_crops += abs(yearly_global_flux_anomaly[i])
    # Adding the grass_crops IAV for that model to the master dictionary.
    all_grass_crops_IAV[file_name] = top_sum_grass_crops / bottom_sum_grass_crops

    # Calculating the sparsely IAV values.
    top_sum_sparsely = 0
    for i in range(52): # Summing for the 52 years.
        top_sum_sparsely += (yearly_sparsely_flux_anomaly[i] * abs(yearly_global_flux_anomaly[i])) / yearly_global_flux_anomaly[i]
    bottom_sum_sparsely = 0
    for i in range(52): # Summing for the 52 years.
        bottom_sum_sparsely += abs(yearly_global_flux_anomaly[i])
    # Adding the sparsely IAV for that model to the master dictionary.
    all_sparsely_IAV[file_name] = top_sum_sparsely / bottom_sum_sparsely

#Updating the IAV values to be a percentage
for region in all_tropical_IAV:
    all_tropical_IAV[region] = all_tropical_IAV[region] * 100
    all_extra_tropical_IAV[region] = all_extra_tropical_IAV[region] * 100
    all_dryland_IAV[region] = all_dryland_IAV[region] * 100
    all_arctic_shrub_IAV[region] = all_arctic_shrub_IAV[region] * 100
    all_grass_crops_IAV[region] = all_grass_crops_IAV[region] * 100
    all_sparsely_IAV[region] = all_sparsely_IAV[region] * 100


#Compiling into a list for creation of the boxplot below
compiled_IAV_data = [list(all_tropical_IAV.values()),
    list(all_extra_tropical_IAV.values()), list(all_dryland_IAV.values()), list(all_arctic_shrub_IAV.values()), 
    list(all_grass_crops_IAV.values()), list(all_sparsely_IAV.values())]

In [ ]:
#This block producing the boxplot of the results

fig, ax = plt.subplots(figsize=(40, 20))

boxplots = ax.boxplot( compiled_IAV_data, widths=0.8, patch_artist=True, showmeans=True, 
    meanprops={"marker": "D", "markerfacecolor": "grey", "markeredgecolor": "black", "markersize": 30}, 
    flierprops=dict(marker="o", markersize=30, markeredgewidth=8), whiskerprops=dict(linewidth=5), capprops=dict(linewidth=5),  
    boxprops=dict(linewidth=5)  
)

ax.set_xticklabels(['Tropical\nForest', 'Extra-Tropical\nForest', 'Semi-Arid', 'Arctic Shrub\n& Tundra', 'Crops &\nGrassland', 'Sparsely\nVegetated'], fontsize=50)

colors = ['limegreen', 'darkgreen', 'darkorange', 'grey', 'slateblue', 'black']

boxes = boxplots['boxes']
whiskers = boxplots['whiskers']
caps = boxplots['caps']
fliers = boxplots['fliers']
medians = boxplots['medians']

i = 0
for box in boxes:
    box.set_facecolor('white')
    box.set_edgecolor(colors[i])
    box.set_linewidth(15)  
    i+=1

i = 0
for median in medians:
    median.set_color(colors[i])
    median.set(linewidth=6) 
    i+=1


i = 0
j=0
for whisker in whiskers:
    whisker.set_color(colors[i])
    j+=1
    if (j % 2 == 0):
        i+=1

i = 0
j=0
for cap in caps:
    cap.set_color(colors[i])
    j+=1
    if (j % 2 == 0):
        i+=1

fliers[1].set(markerfacecolor = 'white', markeredgecolor=colors[1])

#Adding labels
plt.xlabel('\nBiome Definition Scheme', fontsize=65, fontweight='bold')
plt.ylabel('IAV Contribution (%)', fontsize=65, fontweight='bold')
plt.tick_params(axis='y', labelsize=65, width=20)
plt.yticks(fontsize=60, fontweight='bold')
plt.tick_params(axis='x', width=40)
plt.title("(b)", fontsize=70, loc='left', fontweight='bold')
plt.ylim(-5, 55)
#Adding the lettering to denote significance between the biomes
plt.text(1, max(all_tropical_IAV.values())+1, 'A', dict(size=60, weight='bold'), horizontalalignment='center')
plt.text(2, max(all_extra_tropical_IAV.values())+1, 'B', dict(size=70, weight='bold'), horizontalalignment='center')
plt.text(3, max(all_dryland_IAV.values())+1, 'C', dict(size=70, weight='bold'), horizontalalignment='center')
plt.text(4, max(all_arctic_shrub_IAV.values())+1, 'D', dict(size=70, weight='bold'), horizontalalignment='center')
plt.text(5, max(all_grass_crops_IAV.values())+1, 'C', dict(size=70, weight='bold'), horizontalalignment='center')
plt.text(6, max(all_sparsely_IAV.values())+1, 'E', dict(size=70, weight='bold'), horizontalalignment='center')
#Show plot
plt.show()


In [ ]:
#This block producing the barplot of the results

# Fig-Size & bar spacing
plt.subplots(figsize=(24, 10))
width = 0.5
x = np.arange(len(all_tropical_IAV.keys())) * 3.5

#Bar plots
plt.bar(x - 5*width/2, all_tropical_IAV.values(), width=width, label='Tropical Forest', color="limegreen")
plt.bar(x - 3*width/2,  all_extra_tropical_IAV.values(), width=width, label='Extra-Tropical Forest', color="darkgreen")
plt.bar(x - width/2,  all_dryland_IAV.values(), width=width, label='Semi-Arid', color="darkorange")
plt.bar(x + width/2, all_arctic_shrub_IAV.values(), width=width, label='Arctic Shrub & Tundra', color="grey")
plt.bar(x + 3*width/2, all_grass_crops_IAV.values(), width=width, label='Crops & Grassland', color="slateblue")
plt.bar(x + 5*width/2, all_sparsely_IAV.values(), width=width, label='Sparsely Vegetated', color="black")

#Setting the labels
plt.xlabel('Model',fontsize=26)
plt.ylabel('IAV Contribution (%)',fontsize=26)

#Setting the proper lablels for the models
new_labels = []
for key in all_tropical_IAV.keys():
    tempString = str(key)
    tempString = tempString.replace("_s3", "")
    tempString = tempString.upper()
    tempString = tempString.replace("_", "-")
    new_labels.append(tempString)
plt.xticks(x, new_labels, fontsize=22, rotation =45)
plt.yticks(fontsize=22)
plt.legend(fontsize=22, ncols = 2)

# Show plot
plt.show()


In [ ]:
#Getting the specific IAV values and the IQRs for the biomes above
#Compiling into a list for creation of the boxplot below
compiled_IAV_data = [list(all_tropical_IAV.values()),
    list(all_extra_tropical_IAV.values()), list(all_dryland_IAV.values()), list(all_arctic_shrub_IAV.values()), 
    list(all_grass_crops_IAV.values()), list(all_sparsely_IAV.values())]


print(f"Tropical: {round(stats.median(all_tropical_IAV.values()), 2)}")
print(f"Tropical 25: {round(np.percentile(np.array(list(all_tropical_IAV.values())), 25), 2)}")
print(f"Tropical 75: {round(np.percentile(np.array(list(all_tropical_IAV.values())), 75), 2)}")

print(f"\nExtra Tropical: {round(stats.median(all_extra_tropical_IAV.values()), 2)}")
print(f"Extra Tropical 25: {round(np.percentile(np.array(list(all_extra_tropical_IAV.values())), 25), 2)}")
print(f"Extra Tropical 75: {round(np.percentile(np.array(list(all_extra_tropical_IAV.values())), 75), 2)}")

print(f"\nDryland: {round(stats.median(all_dryland_IAV.values()), 2)}")
print(f"Dryland 25: {round(np.percentile(np.array(list(all_dryland_IAV.values())), 25), 2)}")
print(f"Dryland 75: {round(np.percentile(np.array(list(all_dryland_IAV.values())), 75), 2)}")

print(f"\nArctic Shrub: {round(stats.median(all_arctic_shrub_IAV.values()), 2)}")
print(f"Arctic Shrub 25: {round(np.percentile(np.array(list(all_arctic_shrub_IAV.values())), 25), 2)}")
print(f"Arctic Shrub 75: {round(np.percentile(np.array(list(all_arctic_shrub_IAV.values())), 75), 2)}")

print(f"\nGrass & Crops: {round(stats.median(all_grass_crops_IAV.values()), 2)}")
print(f"Grass & Crops 25: {round(np.percentile(np.array(list(all_grass_crops_IAV.values())), 25), 2)}")
print(f"Grass & Crops 75: {round(np.percentile(np.array(list(all_grass_crops_IAV.values())), 75), 2)}")

print(f"\nSparse: {round(stats.median(all_sparsely_IAV.values()), 2)}")
print(f"Sparse 25: {round(np.percentile(np.array(list(all_sparsely_IAV.values())), 25), 2)}")
print(f"Sparse 75: {round(np.percentile(np.array(list(all_sparsely_IAV.values())), 75), 2)}")

print(f"Sum of all == {stats.mean(all_tropical_IAV.values()) + stats.mean(all_extra_tropical_IAV.values()) + stats.mean(all_dryland_IAV.values()) + stats.mean(all_arctic_shrub_IAV.values()) + stats.mean(all_grass_crops_IAV.values()) + stats.mean(all_sparsely_IAV.values())}")

In [ ]:
#This block handles getting the area that all of the different biome defintions occupy

#Getting the tropical area
combined_maps = np.zeros((360,720))
combined_maps[modis_array == 1] = 1
tropical_area_map = combined_maps*area_file
tropical_area = np.sum(tropical_area_map)

#Getting the extra-tropical area
combined_maps = np.zeros((360,720))
combined_maps[modis_array == 2] = 1
extra_tropical_area_map = combined_maps*area_file
extra_tropical_area = np.sum(extra_tropical_area_map)

#Getting the dryland area
combined_maps = np.zeros((360,720))
combined_maps[modis_array == 3] = 1 
dryland_area_map = combined_maps*area_file
dryland_area = np.sum(dryland_area_map)

#Getting the arctic shrub area
combined_maps = np.zeros((360,720))
combined_maps[modis_array == 4] = 1 
arctic_shrub_area_map = combined_maps*area_file
arctic_shrub_area = np.sum(arctic_shrub_area_map)

#Getting the crops & grass area
combined_maps = np.zeros((360,720))
combined_maps[modis_array == 5] = 1 
grass_crops_area_map = combined_maps*area_file
grass_crops_area = np.sum(grass_crops_area_map)

#Getting the sparse area
combined_maps = np.zeros((360,720))
combined_maps[modis_array == 6] = 1 
sparse_area_map = combined_maps*area_file
sparse_area = np.sum(sparse_area_map)


total_surface_area =tropical_area + extra_tropical_area +  dryland_area + arctic_shrub_area +  grass_crops_area + sparse_area #Summing all the regions


print(f"The proportion tropical forest: {round(tropical_area/total_surface_area*100, 2)}\n")
print(f"The proportion extra tropical forest: {round(extra_tropical_area/total_surface_area*100, 2)}\n")
print(f"The proportion dryland: {round(dryland_area/total_surface_area*100, 2)}\n")
print(f"The proportion arctic shrub: {round(arctic_shrub_area/total_surface_area*100, 2)}\n")
print(f"The proportion crops & grass: {round(grass_crops_area/total_surface_area*100, 2)}\n")
print(f"The proportion sparse: {round(sparse_area/total_surface_area*100, 2)}\n")

In [ ]:
#This block is concerned with collecting the values to be used in the permutation testing
print(f"tropicalForest = {list(all_tropical_IAV.values())}")
print(f"extraTropicalForest = {list(all_extra_tropical_IAV.values())}")
print(f"semiArid = {list(all_dryland_IAV.values())}")
print(f"arcticShrub = {list(all_arctic_shrub_IAV.values())}")
print(f"cropsGrass = {list(all_grass_crops_IAV.values())}")
print(f"sparselyVegetated= {list(all_sparsely_IAV.values())}")